## 导入库

In [ ]:
# -*- coding: utf-8 -*-
from __future__ import annotations

# ── 标准库 ──────────────────────────────────────────────────────────────────
import os
import time
from pathlib import Path
from collections import defaultdict
from datetime import datetime

# ── 第三方 ──────────────────────────────────────────────────────────────────
import numpy as np
import pandas as pd
import polars as pl

import torch
import torch.backends.cudnn as cudnn
import lightning as L
import lightning.pytorch as lp
from torch.utils.data import DataLoader

from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import DeviceStatsMonitor
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, Baseline
from pytorch_forecasting.metrics import MAE, RMSE
from pytorch_forecasting.data.encoders import NaNLabelEncoder
from pytorch_forecasting.data import TorchNormalizer, GroupNormalizer


# 你的工程工具
from pipeline.io import cfg, P, fs, storage_options, ensure_dir_local, ensure_dir_az
from pipeline.stream_input_local import ShardedBatchStream  
from pipeline.wr2 import WR2

# ---- 性能/兼容开关（仅一次）----
os.environ.setdefault("POLARS_MAX_THREADS", str(max(1, os.cpu_count() // 2)))
pl.enable_string_cache()
cudnn.benchmark = True
torch.set_float32_matmul_precision("high")

import time as _t


import warnings
warnings.filterwarnings("ignore")  # avoid printing out absolute paths

def _now() -> str:
    return _t.strftime("%Y-%m-%d %H:%M:%S")
print(f"[{_now()}] imports ok")

## 定义工具函数

In [ ]:
# ───────────────────────────────────────────────────────────────────────────
# 滑动窗划分
def make_sliding_cv_by_days(all_days: np.ndarray, *, n_splits: int, gap_days: int, train_to_val: int):
    all_days = np.asarray(all_days).ravel()
    K, R, G = n_splits, train_to_val, gap_days
    usable = len(all_days) - G
    if usable <= 0 or K <= 0 or R <= 0:
        return []
    V_base, rem = divmod(usable, R + K)
    if V_base <= 0:
        return []
    T = R * V_base
    v_lens = [V_base + 1 if i < rem else V_base for i in range(K)]
    folds, v_lo = [], T + G
    for V_i in v_lens:
        v_hi, tr_hi, tr_lo = v_lo + V_i, v_lo - G, v_lo - G - T
        if tr_lo < 0 or v_hi > len(all_days):
            break
        folds.append((all_days[tr_lo:tr_hi], all_days[v_lo:v_hi]))
        v_lo = v_hi
    return folds


## 初始化参数

In [ ]:
# 读入筛选的特征列
df_ranking_features = pd.read_csv("/mnt/data/js/exp/v1/models/tune/feature_importance__fixed__fixed__mm_full_train__features__fs__1300-1500__cv3-g7-r4__seed42__top1000__1760299442__range1000-1600__range1000-1600__cv2-g7-r4__1760347190.csv")
ls_topk_features = df_ranking_features['feature'].tolist()[:500]
TIME_FEATURES = ["time_bucket", "time_pos", "time_sin", "time_cos"]
TO_Z_INPUTS = [c for c in ls_topk_features if c not in TIME_FEATURES]

In [ ]:
# ========== 1) 统一配置 ==========
G_SYM, G_DATE, G_TIME = cfg["keys"]          # e.g. ("symbol_id","date_id","time_id")
TARGET_COL = cfg["target"]                   # e.g. "responder_6"
WEIGHT_COL = cfg["weight"]                   # 允许为 None

# 训练 & CV 超参
N_SPLITS     = 1
GAP_DAYS     = 0
TRAIN_TO_VAL = 6
ENC_LEN      = 10
DEC_LEN      = 1
PRED_LEN     = DEC_LEN
BATCH_SIZE   = 512
LR           = 1e-5
HIDDEN       = 16
HEADS        = 1
DROPOUT      = 0.1
MAX_EPOCHS   = 30

# 数据路径
PANEL_DIR_AZ   = P("az", cfg["paths"].get("panel_shards", "panel_shards"))

TFT_LOCAL_ROOT = P("local", "tft"); ensure_dir_local(TFT_LOCAL_ROOT)

LOCAL_CLEAN_DIR = f"{TFT_LOCAL_ROOT}/clean"; ensure_dir_local(LOCAL_CLEAN_DIR)
CKPTS_DIR = Path(TFT_LOCAL_ROOT) / "ckpts"; ensure_dir_local(CKPTS_DIR.as_posix())
LOGS_DIR  = Path(TFT_LOCAL_ROOT) / "logs";  ensure_dir_local(LOGS_DIR.as_posix())

print("[config] ready")

## 数据导入

In [ ]:
data_start_date = 1400
data_end_date = 1600

data_paths = fs.glob("az://jackson/js_exp/exp/v1/panel_shards/*.parquet")
data_paths =[f"az://{p}" for p in data_paths]

lf_data = (
    pl.scan_parquet(data_paths, storage_options=storage_options)
    .select([*cfg['keys'], WEIGHT_COL, TARGET_COL, *TIME_FEATURES, *TO_Z_INPUTS])
    .filter(pl.col(G_DATE).is_between(data_start_date, data_end_date, closed="both"))
)
lf_data = lf_data.sort([G_SYM, G_DATE, G_TIME])

## 数据处理

### 添加全局时间序列号

In [ ]:
grid_df = (
    lf_data.select([G_DATE, G_TIME]).unique()
        .sort([G_DATE, G_TIME])
        .with_row_index("time_idx")
        .with_columns(pl.col("time_idx").cast(pl.Int64))
        .collect(streaming=True)
)

In [ ]:
container_prefix = "az://jackson/js_exp/exp/v1/tft/panel_clean_shards"; ensure_dir_az(container_prefix)
chunk_size = 30
for lo in range(data_start_date, data_end_date, chunk_size):
    hi = min(lo + chunk_size - 1, data_end_date)
    print(f"processing date range: {lo} ~ {hi}")
    
    lf_chunk = lf_data.filter(pl.col(G_DATE).is_between(lo, hi, closed="both"))
    
    lf_grid_chunk = (
        grid_df.lazy().filter(pl.col(G_DATE).is_between(lo, hi, closed="both"))
    )
    
    lf_joined = (
        lf_chunk.join(lf_grid_chunk, on=[G_DATE, G_TIME], how="left").sort([G_SYM, "time_idx"])
    )
    
    out_path = f"{container_prefix}/panel_clean_{lo:04d}_{hi:04d}.parquet"
    print(f"writing to: {out_path}")
    
    lf_joined.sink_parquet(
        out_path,
        storage_options=storage_options,
        compression="zstd",
    )
print(f"[{_now()}] all done")

### 导入新数据

In [ ]:
# 重新读入数据
data_paths = fs.glob(f"{container_prefix}/*.parquet")
data_paths = [p if p.startswith("az://") else f"az://{p}" for p in data_paths]
lf_ready = pl.scan_parquet(data_paths, storage_options=storage_options).sort([G_SYM, G_DATE, G_TIME])

# 明确要标准化的列
z_cols = [c for c in TO_Z_INPUTS if c in lf_ready.collect_schema().names()]

# ========== 连续特征清洗 ==========
inf2null_exprs  = [pl.when(pl.col(c).is_infinite()).then(None).otherwise(pl.col(c)).alias(c) for c in z_cols]
ffill_exprs = [pl.col(c).forward_fill().over(G_SYM).fill_null(0.0).alias(c) for c in z_cols]


lf_clean = (
    lf_ready.with_columns(inf2null_exprs).with_columns(ffill_exprs) #.with_columns(isna_flag_exprs)
)


## CV 划分

In [ ]:
# ==========  CV 划分 ==========
all_days = (
    lf_clean.select(pl.col(G_DATE)).unique().sort(by=G_DATE)
    .collect(streaming=True)[G_DATE].to_numpy()
)
folds_by_day = make_sliding_cv_by_days(all_days, n_splits=N_SPLITS, gap_days=GAP_DAYS, train_to_val=TRAIN_TO_VAL)
assert len(folds_by_day) > 0, "no CV folds constructed"

## 折内数据标准化处理

In [ ]:

fold_id = 0
# 取第一个 fold 的训练集最后一天，作为统计 z-score 的上界
stats_hi = int(folds_by_day[fold_id][0][-1])
print(f"标准化使用训练集的上限日期 = {stats_hi}")


# ========== Z-score ==========
# 开始计算 z-score
lf_stats_sym = (
    lf_clean.filter(pl.col(G_DATE) <= stats_hi)
            .group_by(G_SYM)
            .agg([pl.col(c).mean().alias(f"mu_{c}") for c in z_cols] +
                [pl.col(c).std(ddof=0).alias(f"std_{c}") for c in z_cols])
)
lf_stats_glb = (
    lf_clean.filter(pl.col(G_DATE) <= stats_hi)
            .select([pl.col(c).mean().alias(f"mu_{c}_glb") for c in z_cols] +
                    [pl.col(c).std(ddof=0).alias(f"std_{c}_glb") for c in z_cols])
)

stats_prefix = "az://jackson/js_exp/exp/v1/tft/stats"; ensure_dir_az(stats_prefix)
# 保存统计结果
lf_stats_sym.collect().write_parquet(
    f"{container_prefix}/fold_{fold_id}_stats_by_symbol.parquet",
    storage_options=storage_options,
    compression="zstd",
)

lf_stats_glb.collect().write_parquet(
    f"{container_prefix}/fold_{fold_id}_stats_by_global.parquet",
    storage_options=storage_options,
    compression="zstd",
)


In [ ]:

lf_z = lf_clean.join(lf_stats_glb, how="cross").join(lf_stats_sym, on=G_SYM, how="left").sort([G_SYM, "time_idx"])

eps = 1e-6
#Z_COLS, NAMARK_COLS = [], [f"{c}__isna" for c in RAW_FEATURES]

Z_COLS = []
for c in z_cols:
    mu_sym, std_sym = f"mu_{c}", f"std_{c}"
    mu_glb, std_glb = f"mu_{c}_glb", f"std_{c}_glb"
    mu_use, std_use = f"{c}_mu_use", f"{c}_std_use"
    z_name = f"{c}_z"

    lf_z = lf_z.with_columns(
        pl.when(pl.col(mu_sym).is_null()).then(pl.col(mu_glb)).otherwise(pl.col(mu_sym)).alias(mu_use),
        pl.when(pl.col(std_sym).is_null() | (pl.col(std_sym) == 0)).then(pl.col(std_glb)).otherwise(pl.col(std_sym)).alias(std_use),
    ).with_columns(
        ((pl.col(c) - pl.col(mu_use)) / (pl.col(std_use) + eps)).alias(z_name)
    ).drop([mu_glb, std_glb, mu_sym, std_sym, mu_use, std_use])

    Z_COLS.append(z_name)

In [ ]:
OUT_COLS = [G_SYM, G_DATE, G_TIME, "time_idx", WEIGHT_COL, TARGET_COL, *TIME_FEATURES] + Z_COLS 
lf_out = lf_z.select(OUT_COLS)

In [ ]:
# 保存lf_out到本地
clean_path_local = P("local", "tft/panel/clean.parquet")
Path(clean_path_local).parent.mkdir(parents=True, exist_ok=True)
lf_out.collect(streaming=True).write_parquet(clean_path_local, compression="zstd")
print(f"[{_now()}] cleaned data saved to {clean_path_local}")


## 训练模型

In [ ]:
# clean 数据导入
clean_path_local = P("local", "tft/panel/clean.parquet")
lf_clean = pl.scan_parquet(clean_path_local)
lf_clean.limit(5).collect()

In [ ]:
KNOWN_REALS = [c for c in lf_clean.collect_schema().names() if c not in (G_SYM, G_DATE, G_TIME, "time_idx", "time_bucket", WEIGHT_COL, TARGET_COL)]

KNOWN_CATEGORIES = ["time_bucket"] if 'time_bucket' in lf_clean.collect_schema().names() else []

UNSCALE_COLS = KNOWN_REALS

TRAIN_COLS = [c for c in lf_clean.collect_schema().names() if c not in (G_DATE, G_TIME)]

# 定义 identity scalers
identity_scalers = {name: None for name in UNSCALE_COLS}

## try

In [ ]:
# 取第一折先试探一下
best_ckpt_paths, fold_metrics = [], []
fold_id = 0
train_days, val_days = folds_by_day[0]

print(f"[fold {fold_id}] train {train_days[0]}..{train_days[-1]} ({len(train_days)} days), "
    f"val {val_days[0]}..{val_days[-1]} ({len(val_days)} days)")

# 明确日期：
train_start_date = int(train_days[0])
train_end_date   = int(train_days[-1])
val_start_date   = int(val_days[0])
val_end_date     = int(val_days[-1])      


# 提取数据
date_range = (train_start_date, val_end_date)
pdf_data = (
    pl.scan_parquet(clean_path_local)
    .filter(pl.col(G_DATE).is_between(train_start_date, val_end_date, closed="both"))
    .collect(streaming=True)
    .to_pandas()
    .sort_values([G_SYM, "time_idx"])
)
pdf_data[G_SYM] = pdf_data[G_SYM].astype("str")
if "time_bucket" in pdf_data.columns:
    pdf_data["time_bucket"] = pdf_data["time_bucket"].astype("str")

# 明确 indexes:
train_end_idx = pdf_data.loc[pdf_data[G_DATE] == train_end_date, "time_idx"].max()
val_start_idx = pdf_data.loc[pdf_data[G_DATE] == val_start_date, "time_idx"].min()
val_end_idx   = pdf_data.loc[pdf_data[G_DATE] == val_end_date, "time_idx"].max()
assert pd.notna(train_end_idx) and pd.notna(val_start_idx) and pd.notna(val_end_idx), "train/val idx not found"
train_end_idx, val_start_idx, val_end_idx = int(train_end_idx), int(val_start_idx), int(val_end_idx)
print(f"[fold {fold_id}] train idx up to {train_end_idx}, val idx {val_start_idx}..{val_end_idx}")  



In [ ]:
t_data = pdf_data[TRAIN_COLS]

identity_scalers = {name: None for name in UNSCALE_COLS}
base_ds = TimeSeriesDataSet(
    t_data,
    time_idx="time_idx",
    target=TARGET_COL,
    group_ids=[G_SYM],
    weight=WEIGHT_COL,
    max_encoder_length=ENC_LEN, 
    min_encoder_length=ENC_LEN,
    max_prediction_length=PRED_LEN, 
    min_prediction_length=PRED_LEN,
    
    static_categoricals=[G_SYM],
    time_varying_known_categoricals=KNOWN_CATEGORIES,
    time_varying_known_reals =KNOWN_REALS,

    categorical_encoders={G_SYM: NaNLabelEncoder(add_nan=True),
                          "time_bucket": NaNLabelEncoder(add_nan=True) if "time_bucket" in KNOWN_CATEGORIES else None,
                          },
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True,
    target_normalizer=GroupNormalizer(
        method="standard", groups=[G_SYM], center=True, scale_by_group=False),
    scalers=identity_scalers,
)

In [ ]:
# 划分训练集，验证集
train_ds = base_ds.filter(
    lambda idx: (
        idx.time_idx_last <= train_end_idx
    ),
    copy=True
)

val_ds = base_ds.filter(
    lambda idx: (
        (idx.time_idx_first_prediction == val_start_idx + ENC_LEN) &
        
        (idx.time_idx_last <= val_end_idx)
    ),
    copy=True
)

In [ ]:
# 数据集加载

train_loader = train_ds.to_dataloader(
    train=True, 
    batch_size=BATCH_SIZE, 
    num_workers=14,
    pin_memory=True,
    persistent_workers=False,
    prefetch_factor=8,
)

n_train_batches = len(train_loader)
print(f"[debug] train_loader batches = {n_train_batches}")
assert n_train_batches > 0, "Empty train dataloader. Check min_prediction_idx/ENC_LEN/date windows."

val_loader = val_ds.to_dataloader(
    train=False,
    batch_size=BATCH_SIZE,
    num_workers=14,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=8,
)

n_val_batches = len(val_loader)
print(f"[debug] val_loader batches = {n_val_batches}")
assert n_val_batches > 0, "Empty val dataloader. Check min_prediction_idx/ENC_LEN/date windows."

In [ ]:
lp.seed_everything(42)
trainer = lp.Trainer(
    accelerator="gpu",
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1,
)

tft = TemporalFusionTransformer.from_dataset(
    train_ds,
    # not meaningful for finding the learning rate but otherwise very important
    learning_rate=LR,
    hidden_size=HIDDEN,  # most important hyperparameter apart from learning rate
    # number of attention heads. Set to up to 4 for large datasets
    attention_head_size=HEADS,
    dropout=DROPOUT,  # between 0.1 and 0.3 are good values
    hidden_continuous_size=HIDDEN // 2,  # set to <= hidden_size
    loss=RMSE(),
    optimizer=torch.optim.Adam,
    # reduce learning rate if no improvement in validation loss after x epochs
    # reduce_on_plateau_patience=1000,
)
print(f"Number of parameters in network: {tft.size() / 1e3:.1f}k")
    
# find optimal learning rate
from lightning.pytorch.tuner import Tuner

res = Tuner(trainer).lr_find(
    tft,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
    max_lr=10.0,
    min_lr=1e-6,
)

print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()

## main

In [ ]:
lp.seed_everything(42) 

# ========== 8) 训练（按 CV 折） ========== 先取第一折
best_ckpt_paths, fold_metrics = [], []

#for fold_id, (train_days, val_days) in enumerate(folds_by_day, start=1):
####################################
fold_id = 0
train_days, val_days = folds_by_day[fold_id]
####################################

print(f"[fold {fold_id}] train {train_days[0]}..{train_days[-1]} ({len(train_days)} days), "
    f"val {val_days[0]}..{val_days[-1]} ({len(val_days)} days)")

# 明确日期：
train_start_date = int(train_days[0])
train_end_date   = int(train_days[-1])
val_start_date   = int(val_days[0])
val_end_date     = int(val_days[-1])      

# 提取数据
date_range = (train_start_date, val_end_date)
pdf_data = (
    lf_clean
    .filter(pl.col(G_DATE).is_between(train_start_date, val_end_date, closed="both"))
    .collect(streaming=True)
    .to_pandas()
) 

pdf_data[G_SYM] = pdf_data[G_SYM].astype("str")
if "time_bucket" in pdf_data.columns:
    pdf_data["time_bucket"] = pdf_data["time_bucket"].astype("str")
pdf_data.sort_values([G_SYM, "time_idx"], inplace=True)

In [ ]:

# 明确 indexes:
train_end_idx = pdf_data.loc[pdf_data[G_DATE] == train_end_date, "time_idx"].max()
val_start_idx = pdf_data.loc[pdf_data[G_DATE] == val_start_date, "time_idx"].min()
val_end_idx   = pdf_data.loc[pdf_data[G_DATE] == val_end_date, "time_idx"].max()
assert pd.notna(train_end_idx) and pd.notna(val_start_idx) and pd.notna(val_end_idx), "train/val idx not found"
train_end_idx, val_start_idx, val_end_idx = int(train_end_idx), int(val_start_idx), int(val_end_idx)
print(f"[fold {fold_id}] train idx up to {train_end_idx}, val idx {val_start_idx}..{val_end_idx}")  


In [ ]:
# 构建训练集 timeseries dataset

t_data = pdf_data[TRAIN_COLS]
train_df = t_data.loc[t_data["time_idx"] <= train_end_idx]

train_ds = TimeSeriesDataSet(
    train_df,
    time_idx="time_idx",
    target=TARGET_COL,
    group_ids=[G_SYM],
    weight=WEIGHT_COL,
    max_encoder_length=ENC_LEN, min_encoder_length=ENC_LEN,
    max_prediction_length=PRED_LEN, min_prediction_length=PRED_LEN,
    time_varying_known_reals =KNOWN_REALS,
    time_varying_known_categoricals=KNOWN_CATEGORIES,
    static_categoricals=[G_SYM],
    categorical_encoders={G_SYM: NaNLabelEncoder(add_nan=True),
                        "time_bucket": NaNLabelEncoder(add_nan=True) if "time_bucket" in pdf_data.columns else None},
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True,
    target_normalizer=GroupNormalizer(
        method="standard", groups=[G_SYM], center=True, scale_by_group=False),
    scalers=identity_scalers,
)

In [ ]:

# 验证集复用 train_ds 的所有 encoders/normalizer（不泄漏）

val_df = t_data.loc[t_data["time_idx"].between(val_start_idx, val_end_idx, inclusive="both")]
val_ds = TimeSeriesDataSet.from_dataset(
    train_ds,
    val_df,
    min_prediction_idx=val_start_idx+ENC_LEN,
    stop_randomization=True,
    predict=False
)

In [ ]:
# 数据集加载

train_loader = train_ds.to_dataloader(
    train=True, 
    batch_size=BATCH_SIZE, 
    num_workers=14,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=4,
    shuffle=True
)

n_train_batches = len(train_loader)
print(f"[debug] train_loader batches = {n_train_batches}")
assert n_train_batches > 0, "Empty train dataloader. Check min_prediction_idx/ENC_LEN/date windows."

val_loader = val_ds.to_dataloader(
    train=False,
    batch_size=BATCH_SIZE*4,
    num_workers=14,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=4,
    shuffle=False,
)

n_val_batches = len(val_loader)
print(f"[debug] val_loader batches = {n_val_batches}")
assert n_val_batches > 0, "Empty val dataloader. Check min_prediction_idx/ENC_LEN/date windows."

In [ ]:
# 8.6 callbacks/logger/trainer
ckpt_dir_fold = Path(CKPTS_DIR) / f"fold_{fold_id}"
ckpt_dir_fold.mkdir(parents=True, exist_ok=True)

callbacks = [EarlyStopping(monitor="val_loss", mode="min", patience=5, check_on_train_epoch_end=False),
            ModelCheckpoint(monitor="val_loss", mode="min", save_top_k=1, dirpath=ckpt_dir_fold.as_posix(), filename=f"fold{fold_id}-tft-best-{{epoch:02d}}-{{val_loss:.5f}}", save_on_train_epoch_end=False),
            LearningRateMonitor(logging_interval="step"),]
RUN_NAME = (f"f{fold_id}"f"_E{MAX_EPOCHS}"f"_lr{LR}"f"_bs{BATCH_SIZE}"f"_enc{ENC_LEN}_dec{DEC_LEN}"f"_{datetime.now().strftime('%Y%m%d-%H%M%S')}")
logger = TensorBoardLogger(save_dir=LOGS_DIR.as_posix(),name="tft",version=RUN_NAME,default_hp_metric=False)

trainer = lp.Trainer(max_epochs=3,
                    accelerator="gpu",
                    devices=1,
                    precision="bf16-mixed",
                    enable_model_summary=True,
                    gradient_clip_val=1.0,
                    gradient_clip_algorithm="norm",
                    fast_dev_run=False,
                    limit_train_batches=50,
                    limit_val_batches=25,
                    val_check_interval=0.25,
                    num_sanity_val_steps=2,
                    log_every_n_steps=10,
                    callbacks=callbacks,
                    logger=logger,
                    accumulate_grad_batches=1,
                    deterministic=False
                    )

tft = TemporalFusionTransformer.from_dataset(
    train_ds,
    learning_rate=LR,
    hidden_size=HIDDEN,
    attention_head_size=HEADS,
    dropout=DROPOUT,
    hidden_continuous_size=HIDDEN // 2,
    loss=RMSE(),
    logging_metrics=[RMSE()],
    optimizer=torch.optim.AdamW,
    optimizer_params={"weight_decay": 1e-4},
    reduce_on_plateau_patience=4,
)
trainer.fit(
    tft,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
    )

In [ ]:
ckpt_cb = next(cb for cb in callbacks if isinstance(cb, ModelCheckpoint))
best_path = ckpt_cb.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_path)

predictions = best_tft.predict(
    val_loader,
    return_y=True,
    trainer_kwargs=dict(accelerator="gpu")
)
y_pred = predictions.output
y_true, w = predictions.y

In [ ]:
num = (w * (y_true - y_pred).pow(2)).sum()
den = (w * y_true.pow(2)).sum()

wr2 = 1.0 - num / (den + eps)
print(f"wr2 after training: {wr2.item():.6f}")

In [ ]:
val_ds.get_parameters

In [ ]:
num = (torch.square(y_true - y_pred) * w).sum()
den = (torch.square(y_true) * w).sum()  
wr2 = 1 - num / den
print(f"wr2 after training: {wr2}")